# Recommender

In [1]:
import pandas as pd
# import datetime as dt

#### Import Data

In [2]:
df = pd.read_csv("KaDo_Timestamps.csv", sep=",")

# df["timestamps"] = df["MOIS_VENTE"].apply(lambda x: dt.datetime.strptime("2020-{}-01 12:50:00".format(x), "%Y-%m-%d %H:%M:%S").timestamp())

df.head()

,TICKET_ID,MOIS_VENTE,PRIX_NET,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,timestamps
0,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.601549e+09
1,35592159,10,1.66,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.601549e+09
2,35592159,10,7.45,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,1.601549e+09
3,35592159,10,5.95,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,1.601549e+09
4,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,1.601549e+09


In [4]:
# df.to_csv("KaDo_Timestamps.csv", sep=",", index=False)

#### Define categories for categorization

In [3]:
CATEGORIES = []

CATEGORIES.extend(df["FAMILLE"].unique().tolist())
CATEGORIES.extend(df["MAILLE"].unique().tolist())
CATEGORIES.extend(df["UNIVERS"].unique().tolist())
CATEGORIES.extend(df["TICKET_ID"].unique().tolist())
CATEGORIES.extend(df["LIBELLE"].unique().tolist())

CATEGORIES = list(dict.fromkeys(CATEGORIES))

CATEGORIES[:5]

['HYGIENE', 'SOINS DU VISAGE', 'PARFUMAGE', 'SOINS DU CORPS', 'MAQUILLAGE']

#### Categorization of string to convert them to int

In [4]:
# df["FAMILLE"] = pd.Categorical(df["FAMILLE"], categories=CATEGORIES)
# df["FAMILLE"] = df["FAMILLE"].cat.codes

# df["UNIVERS"].cat = pd.Categorical(df["UNIVERS"],
#                                        categories=CATEGORIES)
# df["UNIVERS"] = df["UNIVERS"].cat.codes

# df["MAILLE"] = pd.Categorical(df["MAILLE"], categories=CATEGORIES)
# df["MAILLE"] = df["MAILLE"].cat.codes

df["ITEM_ID"] = pd.Categorical(df["LIBELLE"], categories=CATEGORIES)
df["ITEM_ID"] = df["ITEM_ID"].cat.codes

df.head()

,TICKET_ID,MOIS_VENTE,PRIX_NET,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,timestamps,ITEM_ID
0,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.601549e+09,2734987
1,35592159,10,1.66,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,1.601549e+09,2734987
2,35592159,10,7.45,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,1.601549e+09,2734988
3,35592159,10,5.95,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,1.601549e+09,2734989
4,35592159,10,1.67,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,1.601549e+09,2734990


### Products

In [5]:
# df_products = df.copy()
# df_products = df_products[["LIBELLE", "ITEM_ID"]]

# df_products.drop_duplicates(inplace=True)

# df_products.to_csv("Products_ids.csv", sep=",", index=False)

# df_products.head()


,LIBELLE,ITEM_ID
0,GD JDM4 PAMPLEMOUSSE FL 200ML,2734987
2,CR JR PARF BIO.SPE AC.SENT.50ML,2734988
3,EAU MICELLAIRE 3 THES FL200ML,2734989
4,GD JDM4 TIARE FL 200ML,2734990
5,EDT UN MATIN AU JARDIN 100ML MUGUET,2734991


# SPOTLIGHT

## Interactions

In [8]:
from spotlight.interactions import Interactions

implicit_interactions = Interactions(df["CLI_ID"].values, df["ITEM_ID"].values, timestamps=df["timestamps"])

if 0 in implicit_interactions.item_ids:
    print("0000000000000000")

In [9]:
from spotlight.sequence.implicit import ImplicitSequenceModel

model = ImplicitSequenceModel()

interactions = implicit_interactions.to_sequence()

# implicit_model.predict(df["CLI_ID"].unique(), item_ids=None)

In [10]:
model.fit(interactions, verbose=True)

Epoch 0: loss 0.2582200442479081
Epoch 1: loss 0.08467396091673723
Epoch 2: loss 0.06336310845578935
Epoch 3: loss 0.05681315611015507
Epoch 4: loss 0.05205912045247311
Epoch 5: loss 0.047719719895290245
Epoch 6: loss 0.043713032796554994
Epoch 7: loss 0.03994667960607591
Epoch 8: loss 0.036449134806357524
Epoch 9: loss 0.03320268492231065


In [11]:
import pickle

pickle.dump(model, open("models/model_spotlight.plk", "wb"))

In [13]:
import numpy as np
np.argmax(model.predict([2734987, 2734987]))

2735111